# Task - simple text search

* write a function (wraper over native pyspark dataframe functions) that detects if a given name is contained in the question text
* this function should return one of these names (if they are contained in the text): einstein, newton, maxwell, dirac, gauss. If non of them is contained in the text return other
* Write the function in such way it can handle large array of words
* We will use this function again in a streaming application later on

Note:
* In this ntb you will build programatically column expression and do a simple search in text

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, array, lit, when

import os

In [2]:
spark = (
    SparkSession
    .builder
    .appName('Text search')
    .getOrCreate()
)

In [3]:
base_path = os.getcwd()

project_path = ('/').join(base_path.split('/')[0:-3]) 

questions_input_path = os.path.join(project_path, 'output/questions-transformed')

In [4]:
questionsDF = (
    spark
    .read
    .option('path', questions_input_path)
    .load()
)

#### Implement function for text search

Hint
* use when-otherwise condition
* use like function for text search
* the function should take column as input and return another column as output

In [5]:
def get_person(message):
    return (
        when(message.like('%einstein%'), 'einstein')
        .when(message.like('%newton%'), 'newton')
        .when(message.like('%maxwell%'), 'maxwell')
        .when(message.like('%dirac%'), 'dirac')
        .when(message.like('%gauss%'), 'gauss')
        .otherwise('other')
    )

In [7]:
def get_category(message):
    return (
        when(message.like('%quantum physics%'), 'qp')
        .when(message.like('%relativity%'), 'rel')
        .when(message.like('%theoretical mechanics%'), 'tm')
        .when(message.like('%thermodynamics%'), 'thdm')
        .when(message.like('%cosmology%'), 'cosm')
        .otherwise('other')
    )

In [8]:
(
    questionsDF
    .withColumn('physicist', get_category(col('body')))
    .groupBy('physicist')
    .agg(count('*'))
    .orderBy('physicist')
).show(truncate=False)

+---------+--------+
|physicist|count(1)|
+---------+--------+
|cosm     |420     |
|other    |148343  |
|qp       |406     |
|rel      |4150    |
|thdm     |1576    |
|tm       |10      |
+---------+--------+



#### Apply the function

* use also groupBy the result to see how many occurences are there for each name

In [6]:
(
    questionsDF
    .withColumn('physicist', get_person(col('body')))
    .groupBy('physicist')
    .agg(count('*'))
    .orderBy('physicist')
).show(truncate=False)

+---------+--------+
|physicist|count(1)|
+---------+--------+
|dirac    |213     |
|einstein |181     |
|gauss    |428     |
|maxwell  |156     |
|newton   |588     |
|other    |153339  |
+---------+--------+



#### Now implement the function more dynamically

Hint:
* define a list of names that we look for
* iterate over the array and build the condition that is used in the function

In [ ]:
names = ['einstein', 'newton', 'maxwell', 'dirac', 'gauss']

In [ ]:
def get_person_dynamic(message):
    col_exp = when(lit(False), '')
    for name in names:
        col_exp = col_exp.when(message.like('%{}%'.format(name)), name)
    return col_exp.otherwise('other')

In [ ]:
(
    questionsDF
    .withColumn('physicist', get_person_dynamic(col('body')))
    .groupBy('physicist')
    .agg(count('*'))
    .orderBy('physicist')
).show(truncate=False)

In [ ]:
spark.stop()

Note
* other possibilities for text search are using the functions:
    * rlike
    * regexp_extract